This notebook continuously estimates the prevailing, realtime, positivity values via 

* $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$

curves, wherein

$\qquad \qquad \qquad positives_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{P_{\delta}}  \right) \times \frac{1}{population}$


$\qquad \qquad \qquad \qquad  tests_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{T_{\delta}} \right) \times \frac{1}{population}$  

<br>

which are the positive cases per hundred thousand and test numbers per hundred thousand, respectively, based on their cumulative values and

<br>

<table style="width:35%; text-align: left; border: 0px solid black; float:left; margin-left: 50px">
    <tr>
        <th style="width:20%;">$\mathcal{Variable} \qquad$</th><th></th> 
    </tr>
    <tr>
        <td>$\tau$</td><td>end date</td>
    </tr>
    <tr>
        <td>$\delta$</td><td>a series date</td>
    </tr>
    <tr>
      <td>$P_{\delta}$</td><td>The number of positive cases on date $\delta$.</td>
    </tr>
    <tr>
      <td>$T_{\delta}$</td><td>The number of tests on date $\delta$.</td>
    </tr>
    
</table>


<br>

<br>

The subscript $c$ denotes cumulative.   The graph image below illustrates the curves in question.  In brief 

* The quotient of the coordinates of a point along a curve is the cumulative positivity value. 

* The gradient of a modelled secant line to a sequential set of curve rear points estimates the prevailing positivity value.

* The aforementioned secant line model predicts the expected values of $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}}$ increases.


<div style="margin-left: 100px;">
    <img src="https://github.com/briefings/sars/raw/develop/fundamentals/atlantic/docs/ptr.png" longdesc="surveillance" width="40%" style="float:center">
</div>

<br>

Upcoming: Link this image to the interactive graph.



<br>

## Preliminaries

### Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import dask

import logging

import os
import pathlib
import sys

import statsmodels.regression.linear_model


In [2]:
from statsmodels.formula.api import ols

<br>

### Paths

In [3]:
child = os.getcwd()
parent = str(pathlib.Path(child).parent)

In [4]:
root = os.path.join(child, 'warehouse')
warehouse = os.path.join(root, 'prospects')

<br>

Appending paths

In [5]:
sys.path.append(parent)

<br>

### Logging

In [6]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>

### Custom

Import

In [7]:
import cartographs.boundaries.us.boundaries
import cartographs.boundaries.us.settings

In [8]:
import algorithms.curves.estimates
import algorithms.curves.predictions
import algorithms.curves.secants

In [9]:
import atlantic.base.directories
import atlantic.investigations.prospects
import atlantic.investigations.hospitalizations

<br>

Set-up directories

In [10]:
directories = atlantic.base.directories.Directories()
directories.cleanup(listof=[warehouse])
directories.create(listof=[warehouse])

<br>
<br>

### Classes

**Boundaries**

In [11]:
settings = cartographs.boundaries.us.settings.Settings()
boundaries = cartographs.boundaries.us.boundaries.Boundaries(settings.crs)

In [12]:
states = boundaries.states(year=settings.latest)
states.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     object  
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 4.5+ KB


<br>
<br>

## Data

<br>

**Attributes**

In [13]:
fields = ['datetimeobject', 'STUSPS', 'positiveIncrease', 'testIncrease', 'deathIncrease', 'hospitalizedIncrease', 
          'positiveCumulative', 'testCumulative',  'deathCumulative', 'hospitalizedCumulative', 
          'positiveRate', 'testRate', 'deathRate', 'hospitalizedRate', 'ndays']

dtype = {'STUSPS': 'str', 'positiveIncrease': np.float64, 'testIncrease': np.float64, 'deathIncrease': np.float64, 
         'hospitalizedIncrease': np.float64, 'positiveCumulative': np.float64, 'testCumulative': np.float64, 
         'deathCumulative': np.float64, 'hospitalizedCumulative': np.float64, 'positiveRate': np.float64, 
         'testRate': np.float64, 'deathRate': np.float64, 'hospitalizedRate': np.float64, 'ndays': np.int64}        

parse_dates = ['datetimeobject']

<br>
<br>

**The URL strings of the sources**

In [14]:
uribaseline = os.path.join(parent, 'warehouse', 'baselines.csv')
urihospitalizations = os.path.join(parent, 'warehouse', 'hospitalizations.csv')

<br>
<br>

**Baseline**

In [15]:
baselines = pd.read_csv(filepath_or_buffer=uribaseline, header=0, usecols=fields, 
                        dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

baselines.loc[:, 'positiveTestRate'] = np.where(
    baselines['testRate'] > 0, 100 * baselines['positiveRate'] / baselines['testRate'], 0)
baselines.loc[:, 'deathPositiveRate'] = np.where(
    baselines['positiveRate'] > 0, 100 * baselines['deathRate'] / baselines['positiveRate'], 0)

logger.info(baselines.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18096 entries, 0 to 18095
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetimeobject          18096 non-null  datetime64[ns]
 1   STUSPS                  18096 non-null  object        
 2   deathIncrease           18096 non-null  float64       
 3   deathCumulative         18096 non-null  float64       
 4   positiveIncrease        18096 non-null  float64       
 5   positiveCumulative      18096 non-null  float64       
 6   testIncrease            18096 non-null  float64       
 7   testCumulative          18096 non-null  float64       
 8   hospitalizedIncrease    18096 non-null  float64       
 9   hospitalizedCumulative  18096 non-null  float64       
 10  deathRate               18096 non-null  float64       
 11  positiveRate            18096 non-null  float64       
 12  testRate                18096 non-null  float6

<br>
<br>

**Hospitalizations**

In [16]:
hospitalizations = pd.read_csv(filepath_or_buffer=urihospitalizations, header=0, 
                               usecols=fields,dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

hospitalizations.loc[:, 'hospitalizedPositiveRate'] =  np.where(
    hospitalizations['positiveRate'] > 0, 100 * hospitalizations['hospitalizedRate'] / hospitalizations['positiveRate'], 0)
hospitalizations.loc[:, 'deathHospitalizedRate'] =  np.where(
    hospitalizations['hospitalizedRate'] > 0, 100 * hospitalizations['deathRate'] / hospitalizations['hospitalizedRate'], 0)

logger.info(hospitalizations.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12876 entries, 0 to 12875
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetimeobject            12876 non-null  datetime64[ns]
 1   STUSPS                    12876 non-null  object        
 2   deathIncrease             12876 non-null  float64       
 3   deathCumulative           12876 non-null  float64       
 4   positiveIncrease          12876 non-null  float64       
 5   positiveCumulative        12876 non-null  float64       
 6   testIncrease              12876 non-null  float64       
 7   testCumulative            12876 non-null  float64       
 8   hospitalizedIncrease      12876 non-null  float64       
 9   hospitalizedCumulative    12876 non-null  float64       
 10  deathRate                 12876 non-null  float64       
 11  positiveRate              12876 non-null  float64       
 12  testRate          

<br>
<br>

## Regression

<br>

**Latest Date**

In [17]:
latestdate = baselines.datetimeobject.max()
logger.info(type(latestdate))
logger.info(latestdate)

INFO:__main__:<class 'pandas._libs.tslibs.timestamps.Timestamp'>
INFO:__main__:2021-02-19 00:00:00


<br>
<br>

**Periods**

Within the context of the aforementioned $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$ curves, each value of this array is the number of data points, starting from the end of a curve, that are used to develop **a regression model of the tangent/secant line to the points**.

Key points:

* The model predicts the expected $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}, \tau}$ increases.

* The gradient of each model is an estimate of the prevailing positivity rate; an estimate of the current daily positivity value. [cf. continous positivity value] 

In [18]:
periods = np.arange(8, 29)
logger.info( periods)

INFO:__main__:[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28]


<br>
<br>

**Partitions**

In [19]:
partitionby = 'STUSPS'
partitions = baselines[partitionby].unique()
logger.info(partitions)

INFO:__main__:['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'PR']


<br>
<br>

### **positives vs. tests**

<br>

**Analysis**

In [20]:
formula = 'positiveRate ~ testRate'
regressor = 'testRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=100, num=5, latestdate=latestdate)

In [21]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

**State of affairs**

In [22]:
ptr = estimates

<br>

Prevailing Positive Test Rate

In [23]:
ptr.loc[:, 'prevailingPTR'] = 100 * ptr['gradient']
ptr = ptr.merge(baselines[['datetimeobject', 'STUSPS', 'positiveTestRate']], how='left', on=['datetimeobject', 'STUSPS'])
ptr.loc[:, 'cf'] = ptr[['prevailingPTR', 'positiveTestRate']].max(axis=1)

ptr.loc[:, 'rank'] = ptr['cf'].rank(method='min', ascending=False)
ptr.sort_values(by='rank', ascending=True, inplace=True)
ptr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [24]:
ptr.loc[:, 'prevailingTPC'] = np.where(ptr['prevailingPTR'] > 0, ptr['prevailingPTR'].rdiv(100), 0)
ptr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'positiveTestRate': 'continuousPTR'},
          inplace=True)
ptr.to_csv(path_or_buf=os.path.join(warehouse, 'ptr.csv'), index=False, header=True, encoding='utf-8')
ptr.to_json(path_or_buf=os.path.join(warehouse, 'ptr.json'), orient='values')

ptr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingPTR,continuousPTR,rank,prevailingTPC
1,2021-02-19,0.0203981,0.0191252,0.0212648,5.79267e-26,0.999049,3002.01,AK,2.03981,3.386783,48.0,49.0242
8,2021-02-19,0.029491,0.0197283,0.0313355,1.1304e-17,0.991564,637.338,DC,2.9491,3.324426,49.0,33.9086
19,2021-02-19,0.0239708,0.00528608,0.0310531,2.80684e-11,0.939682,477.635,ME,2.39708,2.801979,50.0,41.7175
11,2021-02-19,0.0137494,0.00945343,0.0198755,1.38577e-14,0.98109,942.024,HI,1.37494,2.635832,51.0,72.7304
45,2021-02-19,0.0192335,0.0137548,0.0229591,7.2022e-18,0.982328,-794.356,VT,1.92335,1.420416,52.0,51.9927


<br>
<br>

### **deaths vs. positives**

In [25]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [26]:
formula = 'deathRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=20, num=5, latestdate=latestdate)

In [27]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

**State of Affairs**

In [28]:
dpr = estimates

<br>

Prevailing Death Positive Rate

In [29]:
dpr.loc[:, 'prevailingDPR'] = 100 * dpr['gradient']
dpr = dpr.merge(baselines[['datetimeobject', 'STUSPS', 'deathPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
dpr.loc[:, 'cf'] = dpr[['prevailingDPR', 'deathPositiveRate']].max(axis=1)

dpr.loc[:, 'rank'] = dpr['cf'].rank(method='min', ascending=False)
dpr.sort_values(by='rank', ascending=True, inplace=True)
dpr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [30]:
dpr.loc[:, 'prevailingCPD'] = np.where(dpr['prevailingDPR'] > 0, dpr['prevailingDPR'].rdiv(100), 0)
dpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'deathPositiveRate': 'continuousDPR'},
          inplace=True)
dpr.to_csv(path_or_buf=os.path.join(warehouse, 'dpr.csv'), index=False, header=True, encoding='utf-8')
dpr.to_json(path_or_buf=os.path.join(warehouse, 'dpr.json'), orient='values')

dpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDPR,continuousDPR,rank,prevailingCPD
45,2021-02-19,0.00771505,0.00354066,0.00910931,5.31055e-12,0.952375,13.4369,VT,0.771505,1.354386,48.0,129.617
46,2021-02-19,0.0111113,0.006574,0.0131841,1.3612e-13,0.97485,10.652,VA,1.11113,1.267665,49.0,89.9986
27,2021-02-19,0.0126045,0.00059042,0.0243239,1.55078e-10,0.927494,-24.5497,NE,1.26045,1.029536,50.0,79.3366
44,2021-02-19,0.00828407,0.00525165,0.0103885,1.60778e-17,0.990252,-37.422,UT,0.828407,0.502115,51.0,120.714
1,2021-02-19,0.00795872,0.00324137,0.0108033,6.08624e-07,0.843852,-20.4513,AK,0.795872,0.523570,52.0,125.648


<br>
<br>

### **hospitalizations vs. positives**

In [31]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [32]:
formula = 'hospitalizedRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=hospitalizations, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [33]:
estimates, predictions = prospects.exc(partitions=hospitalizations[partitionby].unique())

<br>

**State of Affairs**

In [34]:
hpr = estimates

<br>

Prevailing Hospitalised Positive Rate

In [35]:
hpr.loc[:, 'prevailingHPR'] = 100 * hpr['gradient']
hpr.loc[:, 'rank'] = hpr['prevailingHPR'].rank(method='min', ascending=False)
hpr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [36]:
hpr = hpr.merge(hospitalizations[['datetimeobject', 'STUSPS', 'hospitalizedPositiveRate']], 
                how='left', on=['datetimeobject', 'STUSPS'])
hpr.loc[:, 'prevailingCPH'] = np.where(hpr['prevailingHPR'] > 0, hpr['prevailingHPR'].rdiv(100), 0)

hpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'hospitalizedPositiveRate': 'continuousHPR'}, 
           inplace=True)

hpr.to_csv(path_or_buf=os.path.join(warehouse, 'hpr.csv'), index=False, header=True, encoding='utf-8')
hpr.to_json(path_or_buf=os.path.join(warehouse, 'hpr.json'), orient='values')

hpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingHPR,rank,continuousHPR,prevailingCPH
32,2021-02-19,0.0176694,0.00540414,0.0356135,7.79621e-08,0.846246,129.384,MS,1.76694,33.0,3.097015,56.5951
33,2021-02-19,0.0139163,0.010592,0.0193355,3.75334e-13,0.965776,64.7445,AK,1.39163,34.0,2.250082,71.8581
34,2021-02-19,0.00793225,0.00568656,0.00883134,3.2737e-16,0.988792,37.8039,NH,0.793225,35.0,1.500763,126.068
35,2021-02-19,-4.16334e-17,-2.73196e-14,2.84299e-14,0.894762,-629.417,462.615,NY,-4.16334e-15,36.0,5.752703,0
36,2021-02-19,-2.77556e-17,-3.14095e-14,3.09654e-14,0.877525,-243.579,343.787,CT,-2.77556e-15,36.0,4.488100,0


<br>
<br>

### **deaths vs. hospitalizations**

In [37]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [38]:
formula = 'deathRate ~ hospitalizedRate'
regressor = 'hospitalizedRate'

prospects = atlantic.investigations.prospects.Prospects(data=hospitalizations, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [39]:
estimates, predictions = prospects.exc(partitions=hospitalizations[partitionby].unique())

<br>

**State of Affairs**

In [40]:
dhr = estimates

<br>

Prevailing

In [41]:
dhr.loc[:, 'prevailingDHR'] = 100 * dhr['gradient']
dhr.loc[:, 'rank'] = dhr['prevailingDHR'].rank(method='min', ascending=False)
dhr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [42]:
dhr = dhr.merge(hospitalizations[['datetimeobject', 'STUSPS', 'deathHospitalizedRate']], 
                how='left', on=['datetimeobject', 'STUSPS'])
dhr.loc[:, 'prevailingHPD'] = np.where(dhr['prevailingDHR'] > 0, dhr['prevailingDHR'].rdiv(100), 0)

dhr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'deathHospitalizedRate': 'continuousDHR'}, 
           inplace=True)

dhr.to_csv(path_or_buf=os.path.join(warehouse, 'dhr.csv'), index=False, header=True, encoding='utf-8')
dhr.to_json(path_or_buf=os.path.join(warehouse, 'dhr.json'), orient='values')

dhr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDHR,rank,continuousDHR,prevailingHPD
32,2021-02-19,0.248172,0.23328,0.299956,1.4511e-18,0.99545,-13.8047,MD,24.8172,33.0,22.404786,4.02946
33,2021-02-19,0.205633,0.160016,0.26183,4.62662e-15,0.979687,19.5774,CO,20.5633,34.0,25.444786,4.86304
34,2021-02-19,0.199731,0.109093,0.231682,4.04784e-15,0.984281,23.8855,MN,19.9731,35.0,25.275938,5.00673
35,2021-02-19,0.179447,0.024671,0.539854,6.39874e-07,0.813534,2.67436,HI,17.9447,36.0,19.706691,5.57266
36,2021-02-19,0.172488,0.100269,0.209064,1.01111e-16,0.987735,-20.4073,UT,17.2488,37.0,12.752051,5.7975


<br>
<br>

## Clean-up

In [43]:
!rm -rf *.log
!rm -rf *.pdf
!rm -rf states

<br>
<br>

## End

In [44]:
%%bash

date +"%Y-%m-%d %T"

2021-02-20 18:21:36
